In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
from typing import List, Dict, Tuple
import io
import sys
import pickle
import itertools
import datetime
import copy
from tqdm import tqdm
import random
import csv
import json
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math

import torch

import torch.optim.lr_scheduler as lr_scheduler
import torch_geometric
import torch_geometric.transforms as T
import torch_geometric.transforms
import torch_geometric.datasets
import torch_geometric.nn
from torch_geometric.utils import to_networkx

from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore', module='sklearn')

In [2]:
from env_variables import *
from utils_helpers import *

In [3]:
results_output_dir = "./results"
if not os.path.exists(results_output_dir):
    os.makedirs(results_output_dir)

# Bipartite Matching Algorithms

In [10]:
def get_job_params_bipartite():
    data_types = [
                #"Real", 
                "Real_automatic",
                #"../data/synthetic_algo_100_points",
                #"../data/synthetic_algo_200_points",
                #"../data/synthetic_algo_300_points",
                #"../data/synthetic_algo_400_points",
                #"../data/synthetic_algo_500_points",
                #"../data/synthetic_algo_600_points",
                #"../data/synthetic_algo_700_points",
                #"../data/synthetic_algo_800_points",
                #"../data/synthetic_algo_900_points",
                #"../data/synthetic_algo_1000_points"
    ]

    combinations = {
        "data_type_train": data_types,
        "model_type":[
                        "Hopcroft_Karp",
                        "Minimum Weight"
        ],
        "knn_inter_nodes":[
                            #"min",
                            #7,
                            10
        ],
        "knn_inter_nodes_max": [7],
       "knn_intra_nodes":[0],
        "normalize":[
                    False, 
                     #True
                    ],
        "node_feats":[[
            'Y', 
            'X', 
            'Z', 
            'node_type', 
            'ID'
        ]],
        "edge_feats":[[
    'source',
    'target',
     'edge_label',
     'delta_x',
     'delta_y',
     'delta_z',
     'weight',
     'edge_type',
     'angle_orientation_theta',
     'angle_orientation_phi']],
    }


    job_parameters = []


    # Generate all possible combinations of the dictionary values
    for values in itertools.product(*combinations.values()):
        # Generate a dictionary for the combination of values
        job_dict = dict(zip(combinations.keys(), values))
        job_dict["scale_features"] = True if "Real" in job_dict["data_type_train"] else False
        job_parameters.append(job_dict)
    
    return job_parameters

In [11]:
def get_graph_list_bipartite(job_parameters, debug = False):
    #build dataframes
    graph_list_dict_bipartite = {}

    for params in tqdm(job_parameters):

        params_list = [params["data_type_train"], params["knn_inter_nodes"], params["knn_intra_nodes"], 
                                        params["knn_inter_nodes_max"],params["normalize"],
                                        params["scale_features"], str(params["node_feats"]), str(params["edge_feats"])]
        params_list = [str(param_) for param_ in params_list]
        graph_key = "_".join(params_list)

        if graph_key not in graph_list_dict_bipartite:
            graph_list = get_graph_list(params["data_type_train"], params["knn_inter_nodes"], params["knn_intra_nodes"], 
                                            params["knn_inter_nodes_max"], normalize = params["normalize"],
                                            scale_feats = params["scale_features"],
                                            node_feats = params["node_feats"], edge_feats = params["edge_feats"],
                                            shuffle = False)



            graph_list_dict_bipartite[graph_key] = graph_list

    return graph_list_dict_bipartite

In [12]:
def train_bipartite(graph_list_dict_bipartite, job_parameters, debug = False, make_plots = True):
    results_list_mf = []

    for params in job_parameters:
        print("\n\n\n####################################")
        metrics_list_bipartite = []

        params_list = [params["data_type_train"], params["knn_inter_nodes"], params["knn_intra_nodes"], 
                                        params["knn_inter_nodes_max"],params["normalize"],
                                        params["scale_features"], str(params["node_feats"]), str(params["edge_feats"])]
        params_list = [str(param_) for param_ in params_list]
        graph_key = "_".join(params_list)
        graph_list = graph_list_dict_bipartite[graph_key]

        for graph in graph_list:
            graph_id, nodes_df, edges_df, edges_df_knn, k_intra, k_inter = graph.graph_id, graph.nodes_df, graph.edges_df, graph.edges_df_knn, graph.k_intra, graph.k_inter
            print("Graph_ID:",graph_id,"K_INTRA:",k_intra, "K_INTER:",k_inter)

            edges_df_bipartite_graph = edges_df_knn.copy()
            nx_G_knn = nx_build_graph(nodes_df, edges_df_bipartite_graph)

            model_type = params["model_type"]

            #get the bipartite edges_list
            if(model_type=="Hopcroft_Karp"):
                nx_bipartite_edges_list = nx.bipartite.maximum_matching(nx_G_knn)
            elif(model_type=="Eppstein"):
                nx_bipartite_edges_list = nx.bipartite.eppstein_matching(nx_G_knn)
            elif(model_type=="Minimum Weight"):
                nx_bipartite_edges_list = nx.bipartite.minimum_weight_full_matching(nx_G_knn)
            else:
                raise ValueError("Model not implemented!")

            #convert to Dataframe
            edges_df_bipartite = nx_convert_dict_to_edges_df(nx_bipartite_edges_list)
            #apply the pred labels to edges_df_bipartite, taking as input the edges_df_knn
            edges_df_bipartite["edge_label"] = 1
            edges_df_bipartite = apply_edges_df_label(edges_df_bipartite, edges_df_knn)

            metrics_bipartite, edge_labels_string_bipartite = eval_edges_df(edges_df, edges_df_bipartite)
            metrics_list_bipartite.append(metrics_bipartite)

            print(json.dumps(metrics_bipartite, indent = 1))

            #Save results to file
            bipartite_results_array = pred_df_to_csv(edges_df_bipartite, graph.nodes_df_original)
            output_file_dir =  results_output_dir+"/"+params["data_type_train"]+"_"+params["model_type"]+"_" +\
                                    str(params["knn_inter_nodes"])+"_"+str(params["knn_intra_nodes"])+"/"
            if not os.path.exists(output_file_dir):
                os.makedirs(output_file_dir)
            output_file_path = os.path.join(output_file_dir, graph.graph_id)
            array_to_csv(bipartite_results_array, output_file_path)

            if(make_plots):
                fig_bipartite = df_make_plot(nodes_df, edges_df_bipartite, edge_labels_string_bipartite, "Bipartite")
                plt.show()
        
        ####################################
        ### Aggregate Metrics at the end  ##
        ####################################

        metrics_bipartite_aggregated = aggregate_metrics(metrics_list_bipartite)
        print(json.dumps(metrics_bipartite_aggregated,indent = 1))

        result_bipartite = {}
        result_bipartite["aggregated_metrics"] = metrics_bipartite_aggregated
        params["angle_features"] = "NA"
        params["constraints"] = "NA"
        result_bipartite["job_parameters"] = params
        results_list_mf.append(result_bipartite)
    return results_list_mf

In [13]:
def plot_results_bipartite(results_list_bipartite):
    plot_df_mf = plot_table(results_list_bipartite, metrics_dict_entries = [None])
    plot_df_mf = plot_df_mf.sort_values(by=["K Inter", 'Algorithm', 'Data Train', 'Data Test'])
    display(plot_df_mf)
    #plot_df_mf = plot_df_mf.drop(["Data Train", "Data Test", "Scale"], axis=1)
    plot_df_mf = plot_df_mf.drop(["Data Test", "Scale"], axis=1)
    display(plot_df_mf)
    print(plot_df_to_latex(plot_df_mf))
    return plot_df_mf

In [14]:
job_parameters_bipartite = get_job_params_bipartite()

In [15]:
graph_list_dict_bipartite = get_graph_list_bipartite(job_parameters_bipartite, debug = False)

100%|██████████| 2/2 [00:36<00:00, 18.41s/it]


In [16]:
def array_to_csv(array : np.array, csv_path : str, separator = ",", columns_order=['YN', 'XN', 'YG', 'XG', 'ZN', 'ZG']):
    with open(csv_path, 'w') as fp:
        for i in range(len(array)):
            row_array = array[i]
            array_dict = {}
            array_dict["XN"], array_dict["YN"], array_dict["ZN"], array_dict["XG"], array_dict["YG"], array_dict["ZG"] = row_array[0], row_array[1], row_array[2], row_array[3], row_array[4], row_array[5]
            new_array = [array_dict[col] for col in columns_order]
            row = separator.join(str(v) for v in new_array)
            if(i!=(len(array)-1)):#only write \n up to the line before the last line
                row+="\n"
            fp.write(row)
    return

In [17]:
results_list_bipartite = train_bipartite(graph_list_dict_bipartite, job_parameters_bipartite, debug = True, make_plots = False)




####################################
Graph_ID: Crop1.csv K_INTRA: 0 K_INTER: 10
{
 "tp": 0,
 "fp": 69,
 "tn": 621,
 "fn": 0,
 "acc": 0.9,
 "precision": 0.0,
 "recall": 0,
 "f1_score": 0,
 "tp_percent": 0,
 "tp_total_count": 0
}
Graph_ID: Crop2.csv K_INTRA: 0 K_INTER: 10
{
 "tp": 0,
 "fp": 82,
 "tn": 738,
 "fn": 0,
 "acc": 0.9,
 "precision": 0.0,
 "recall": 0,
 "f1_score": 0,
 "tp_percent": 0,
 "tp_total_count": 0
}
Graph_ID: Crop3.csv K_INTRA: 0 K_INTER: 10
{
 "tp": 0,
 "fp": 53,
 "tn": 497,
 "fn": 0,
 "acc": 0.904,
 "precision": 0.0,
 "recall": 0,
 "f1_score": 0,
 "tp_percent": 0,
 "tp_total_count": 0
}
Graph_ID: Crop4.csv K_INTRA: 0 K_INTER: 10
{
 "tp": 0,
 "fp": 47,
 "tn": 443,
 "fn": 0,
 "acc": 0.904,
 "precision": 0.0,
 "recall": 0,
 "f1_score": 0,
 "tp_percent": 0,
 "tp_total_count": 0
}
Graph_ID: Crop5_BC.csv K_INTRA: 0 K_INTER: 10
{
 "tp": 0,
 "fp": 42,
 "tn": 418,
 "fn": 0,
 "acc": 0.909,
 "precision": 0.0,
 "recall": 0,
 "f1_score": 0,
 "tp_percent": 0,
 "tp_total_count": 

In [18]:
output_df = plot_results_bipartite(results_list_bipartite) 

,Algorithm,Data Train,Data Test,Constraints,Normalize,Scale,Node Feat.,Edge Feat.,K Intra,K Inter,...,Accuracy,Precision,Recall,F1-Score,TP Percent,TP Total Count,TP,FP,TN,FN
0,Hopcroft_Karp,Real_automatic,Real_automatic,NA,False,True,"[Y, X, Z, node_type, ID]","[source, target, edge_label, delta_x, delta_y,...",0,10,...,0.906,0.000,0,0,0,0,0,424,4056,0
1,Minimum Weight,Real_automatic,Real_automatic,NA,False,True,"[Y, X, Z, node_type, ID]","[source, target, edge_label, delta_x, delta_y,...",0,10,...,0.906,0.000,0,0,0,0,0,424,4056,0


,Algorithm,Data Train,Constraints,Normalize,Node Feat.,Edge Feat.,K Intra,K Inter,ROC AUC Score,Accuracy,Precision,Recall,F1-Score,TP Percent,TP Total Count,TP,FP,TN,FN
0,Hopcroft_Karp,Real_automatic,NA,False,"[Y, X, Z, node_type, ID]","[source, target, edge_label, delta_x, delta_y,...",0,10,NA,0.906,0.000,0,0,0,0,0,424,4056,0
1,Minimum Weight,Real_automatic,NA,False,"[Y, X, Z, node_type, ID]","[source, target, edge_label, delta_x, delta_y,...",0,10,NA,0.906,0.000,0,0,0,0,0,424,4056,0


\begin{tabular}{llllrllrrlllrrrrrrr}
\toprule
 & Algorithm & Data Train & Constraints & Normalize & Node Feat. & Edge Feat. & K Intra & K Inter & ROC AUC Score & Accuracy & Precision & Recall & F1-Score & TP Percent & TP & FP & TN & FN \\
\midrule
0 & Hopcroft_Karp & Real_automatic & NA & False & ['Y', 'X', 'Z', 'node_type', 'ID'] & ['source', 'target', 'edge_label', 'delta_x', 'delta_y', 'delta_z', 'weight', 'edge_type', 'angle_orientation_theta', 'angle_orientation_phi'] & 0 & 10 & \textbf{NA} & \textbf{0.906} & \textbf{0.000} & \textbf{0} & \textbf{0} & \textbf{0} & \textbf{0} & \textbf{424} & \textbf{4056} & \textbf{0} \\
1 & Minimum Weight & Real_automatic & NA & False & ['Y', 'X', 'Z', 'node_type', 'ID'] & ['source', 'target', 'edge_label', 'delta_x', 'delta_y', 'delta_z', 'weight', 'edge_type', 'angle_orientation_theta', 'angle_orientation_phi'] & 0 & 10 & \textbf{NA} & \textbf{0.906} & \textbf{0.000} & \textbf{0} & \textbf{0} & \textbf{0} & \textbf{0} & \textbf{424} & \textbf{4

In [19]:
def convert_to_final_format(output_df):
    output_df = output_df.drop(["Node Feat.", "Normalize", "K Intra",  "TP Percent","TP Total Count","TP","FP","TN","FN"], axis = 1)
    output_df = output_df.rename(columns={"K Inter": "K", "Edge Feat.":"Angles"})
    if("Angles" in output_df.columns):
        output_df["Angles"] = output_df["Angles"].apply(lambda x: all("angle" not in item for item in x))
    return output_df

final_output_df = convert_to_final_format(output_df)

In [20]:
final_output_df["Data Train"] = final_output_df["Data Train"].apply(lambda x: os.path.basename(x))
final_output_df= final_output_df.drop(["K", "ROC AUC Score", "Constraints", "Angles"],axis=1)
final_output_df = final_output_df[["Data Train", "Algorithm","Accuracy","Precision","Recall","F1-Score"]]
final_output_df = final_output_df.sort_values(by=['Data Train'])
print(plot_df_to_latex(final_output_df))
final_output_df

\begin{tabular}{lllllrr}
\toprule
 & Data Train & Algorithm & Accuracy & Precision & Recall & F1-Score \\
\midrule
0 & Real_automatic & Hopcroft_Karp & \textbf{0.906} & \textbf{0.000} & \textbf{0} & \textbf{0} \\
1 & Real_automatic & Minimum Weight & \textbf{0.906} & \textbf{0.000} & \textbf{0} & \textbf{0} \\
\bottomrule
\end{tabular}



,Data Train,Algorithm,Accuracy,Precision,Recall,F1-Score
0,Real_automatic,Hopcroft_Karp,0.906,0.000,0,0
1,Real_automatic,Minimum Weight,0.906,0.000,0,0
